In [1]:
# Install packages here
# Packages for data processing
import numpy as np
import pandas as pd
import datetime
from sklearn import preprocessing
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
from scipy.sparse import csr_matrix
import scipy as sp


# Packages for modeling
from surprise import Reader
from surprise import Dataset
from surprise import KNNWithMeans
from surprise import KNNBasic
from surprise.model_selection import GridSearchCV
from surprise.model_selection import cross_validate, train_test_split
from surprise import SVD,accuracy
from surprise import SVDpp
from surprise import NMF
from surprise import SlopeOne
from surprise import CoClustering
import heapq


# Packages for model evaluation
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from time import time


# Package to suppress warnings
import warnings
warnings.filterwarnings("ignore")


# Packages for saving models
import pickle



In [2]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv") 
movies_df = pd.read_csv("movies.csv")
tags_df = pd.read_csv("tags.csv")
links_df = pd.read_csv("links.csv")

genome_scores_df = pd.read_csv("genome_scores.csv")
genome_tags_df = pd.read_csv("genome_tags.csv")
imdb_data_df = pd.read_csv("imdb_data.csv")

In [3]:
sample_df = pd.read_csv("sample_submission.csv")

In [4]:
sample_df.head()

,Id,rating
0,1_2011,1.0
1,1_4144,1.0
2,1_5767,1.0
3,1_6711,1.0
4,1_7318,1.0


In [ ]:
data = Dataset.load_from_df(df_train[['userId', 'movieId', 'rating']], Reader())

In [ ]:
train_set, test_set = train_test_split(data, test_size=0.20)

In [ ]:
svd=SVD(n_epochs = 30, n_factors = 200, init_std_dev = 0.05, random_state=42)

In [ ]:
#Fitting the model
svd.fit(train_set)

In [ ]:
# Make predictions
test_pred= svd.test(test_set)

In [ ]:
#Evaluate model performance
rsme_collabo = accuracy.rmse(test_pred,verbose=True)

In [ ]:
test = pd.read_csv("test.csv")
test.sample(10)



In [ ]:
#Predicting the rating for each user and movie
ratings=[]
for x,y in test.itertuples(index=False):
    output=svd.predict(x,y)
    ratings.append(output)
    
output_df=pd.DataFrame(ratings)[['uid','iid','est']]
output_df['ID']=output_df['uid'].astype(str) + '_' + output_df['iid'].astype(str)
output_df=output_df[['ID','est']]
output_df.head()

In [ ]:
#Creating the "results" dataframe and convert to csv
results = pd.DataFrame({"Id":output_df['ID'],"rating": output_df['est']})
results.to_csv("SVD.csv", index=False)

In [ ]:
Team_10_submission = pd.DataFrame({'id': results.Id, 'rating': results.rating})
Team_10_submission.to_csv('submission.csv', index=False)